# Kaggle Spooky Author Challenge
### In this notebook I use the [Kaggle Spooky Author Dataset](https://www.kaggle.com/c/spooky-author-identification) to demonstrate some natural language processing techniques in Python.

In [240]:
import pandas as pd
import re
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

The [Pandas](https://pandas.pydata.org/) Dataframe is a useful data structure for this project and comes with some easy ways to read in CSV files. We will use [NLTK](http://www.nltk.org/) to demonstrate some fundamental NLP processes. NLTK needs to download some components separate from its install but fortunately we should only need to do this once.

## Loading Data

In [241]:
data = pd.read_csv('train.csv', index_col='id')
print(data[:3])

                                                      text author
id                                                               
id26305  This process, however, afforded me no means of...    EAP
id17569  It never once occurred to me that the fumbling...    HPL
id11008  In his left hand was a gold snuff box, from wh...    EAP


We are only using the training dataset for this demonstration. The testing dataset does not have labels as it is meant to be submitted to the online grader. The data comes with a built in index column so we might as well use it to keep our data organized. These are examples of the raw sentences from the data. There are three targets in the form of authors Edgar Allan Poe, H.P Lovecraft, and Marry Shelly.

## Stripping Punctuation

In [242]:
example = data
example['stripped'] = example['text'].apply(lambda x: re.sub(r'[^\w\s]','',x))
print(example[:3]['stripped'])

id
id26305    This process however afforded me no means of a...
id17569    It never once occurred to me that the fumbling...
id11008    In his left hand was a gold snuff box from whi...
Name: stripped, dtype: object


We are going to make a copy of the data to play around with and you will see why in a moment. When using Dataframes we can apply a function to each row element in a particular column by using the "apply" function and a lambda expression. The regular expression "re.sub(r'[^\w\s]','',x)" strips punctuation off of the sentences which is a critical aspect to creating uniform features.

## Creating Tokens

In [243]:
example['tokens'] = example['stripped'].apply(lambda x: nltk.word_tokenize(x))
print(example[:3]['tokens'])

id
id26305    [This, process, however, afforded, me, no, mea...
id17569    [It, never, once, occurred, to, me, that, the,...
id11008    [In, his, left, hand, was, a, gold, snuff, box...
Name: tokens, dtype: object


This expression simply breaks sentences into individual words. These words will become the building blocks of our feature space.

## Transforming to Lowercase

In [244]:
example['lower'] = example['tokens'].apply(lambda x: [word.lower() for word in x])
print(example[:3]['lower'])

id
id26305    [this, process, however, afforded, me, no, mea...
id17569    [it, never, once, occurred, to, me, that, the,...
id11008    [in, his, left, hand, was, a, gold, snuff, box...
Name: lower, dtype: object


Transforming words to lowercase is a common practice in NLP. The primary reasoning for this is that a word at the start of a sentence should be an equivalent feature to the same word occurring later in a sentence, all other things being equal. There may be specific situations where you do not want to do this but it is common for most text classification problems. In this function each row element "x" is no longer a single string but a list due to the tokenization process. For this reason we need a list comprehension to iterate over each list to transform each item to lowercase.

## Removing Stopwords

In [245]:
stopWords = nltk.corpus.stopwords.words('english')
example['stopped'] = example['lower'].apply(lambda x: [word for word in x if word not in stopWords])
print(example[:3]['stopped'])

id
id26305    [process, however, afforded, means, ascertaini...
id17569    [never, occurred, fumbling, might, mere, mistake]
id11008    [left, hand, gold, snuff, box, capered, hill, ...
Name: stopped, dtype: object


Stopwords are generally very common words that tend to not vary in frequency between different classes. Some examples are "as", "if", "is", "the" "a". For most text classification problems removing stopwords is an easy way to cut down on features that do not convey much information and improve the performance of classifiers down the line.

## Stemming

In [246]:
stemmer = nltk.stem.SnowballStemmer("english")
example['stemmed'] = example['stopped'].apply(lambda x: [stemmer.stem(word) for word in x])
print(example[:3]['stemmed'])

id
id26305    [process, howev, afford, mean, ascertain, dime...
id17569           [never, occur, fumbl, might, mere, mistak]
id11008    [left, hand, gold, snuff, box, caper, hill, cu...
Name: stemmed, dtype: object


Stemming is a common process in NLP which removes certain affixes to get to the core of a word and reduce the size of the feature space. You can see how some of these words have been altered such as "afforded" to "afford".

## All in one with Scikit-Learn Vectorizer

In [247]:
def stem_func(text):
    return [stemmer.stem(word) for word in analyzer(text)]

analyzer = CountVectorizer().build_analyzer()

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(
    lowercase=True,
    stop_words=stopWords,
    analyzer=stem_func,
    ngram_range=(1,3),
    min_df=2
)
x_data = vectorizer.fit_transform(data['text'])
print(vectorizer.get_feature_names()[:10])

['aback', 'abandon', 'abaout', 'abas', 'abash', 'abat', 'abbey', 'abbrevi', 'abdic', 'abdul']


So far we have demonstrated each of these operations individually with NLTK to get an idea of what they look like, their flow, and their purpose. Wouldn't it be nice if we could combine these operations into a single efficient command? It turns out we can with Scikit-Learn CountVectorizer. In addition to stripping punctuation, creating tokens, transforming to lowercase, removing stopwords, and stemming, CountVectorizer can also create a range of N-grams and count these patterns. These counts are used to form a numeric feature vector that we can use for machine learning. 

Using an N-gram range of one to three will capture most patterns in the text that are common enough to make decent features for this problem. We use a minimum document frequency of two to ensure words appear at least twice in the corpus to be considered a feature. To incorporate the stemming functionality from NLTK we have to write a short function to override the built in analyzer. As a sanity check we'll print the first ten features to get an idea of what they look like. 

CountVectorizer has a convenient version called TfidfVectorizer which outputs term frequency inverse document frequency instead of raw counts. This can be a nice addition for some classifiers but our best classifier works better with raw counts so we'll stick to that.

## Labels to numeric values

In [248]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_data = le.fit_transform(data['author'])
print(le.classes_)

['EAP' 'HPL' 'MWS']


We have transformed our text into numeric feature vectors but we also need to convert our target labels into numeric values so that they can be used by our classifiers. LabelEncoder will take care of that. Print the classes LabelEncoder creates is helpful for ensuring that we do not have any messy labels and our data is clean.

## Test, Train Splitting

In [249]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.33, random_state=123)

We need to make a training and testing set to evaluate our classifiers. Fortunately, Scikit-Learn has a convenient one liner to do just that.

## Feature Selection

In [250]:
from sklearn.feature_selection import SelectPercentile, chi2
selector = SelectPercentile(score_func=chi2, percentile=90)
x_train = selector.fit_transform(x_train, y_train)
x_test = selector.transform(x_test)

Our feature space is already productive but we can gain a small performance improvement by discard the 10% worst features according to the [chi2 metric](https://en.wikipedia.org/wiki/Chi-squared_distribution). We transformed the training and testing x sets to make sure the dimensionality remains consistent. 

## Selecting Classifiers

In [251]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

names = ["GaussianNB", "MultinomialNB", "BernoulliNB"]

classifiers = [
    GaussianNB(),
    MultinomialNB(),
    BernoulliNB(),
]

For this problem we are going to examine three flavors of [Naive Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier). In this context, Naive Bayes is a simple classifier that looks at the probability of a pattern occurring in a document, relative to the prior probability of a category and the evidence the classifier has trained on. For text classification Naive Bayes is fast and often performs better than more complicated algorithms.

## Defining Metrics

In [252]:
from sklearn.metrics import accuracy_score, f1_score

def metrics(name, real, pred):
    print(name)
    acc = accuracy_score(real, pred)
    f1 = f1_score(real, pred, average='weighted')
    print("ACC: " + str(acc))
    print("F1: " + str(f1))

We are going to use some built in scoring functions from Scikit-Learn to analyze classifier performance. [F1](https://en.wikipedia.org/wiki/F1_score) is a great metric which balances precision and recall. Since this data set has three labels its a good choice to use a "weighted" setting which weights the score based on the prevalence for each category label. A high F1 score derived in this way will reflect that our classifier is not too specific or too general for any one category. 

## Results

In [253]:
for name, classifier, in zip(names, classifiers):
    classifier.fit(x_train.toarray(), y_train)
    predictions = classifier.predict(x_test.toarray())
    metrics(name, y_test, predictions)

GaussianNB
ACC: 0.6935933147632312
F1: 0.6931428017154011
MultinomialNB
ACC: 0.8308573197152584
F1: 0.8308631638197698
BernoulliNB
ACC: 0.8245125348189415
F1: 0.8242845741505971


This loop goes through our classifiers to train, test, and evaluate their performance. 83% is a respectable outcome for a three category NLP classification task. Next time we will examine some advanced NLP techniques such as different variants of stemming vs lemmatization, wrapper based feature selection, and other classifiers.